In [1]:
import os
import pandas as pd
import gc
from IPython.display import display, HTML
import glob
pd.set_option('display.max_colwidth', None)
import lorem
import random
from collections import defaultdict
import csv
import enchant
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
#All CSV files generated from the DBLP XML
## All files and directories ending with .csv:
files = (glob.glob("DXML/*.csv"))
files[0:5]

['DXML\\output_article.csv',
 'DXML\\output_article_header.csv',
 'DXML\\output_author.csv',
 'DXML\\output_author_authored_by.csv',
 'DXML\\output_book.csv']

### CITATION NODE

In [3]:
# View All papers with Citations from cite_has_citation.csv

citations = pd.read_csv('DXML\\output_cite_has_citation.csv',sep=';')
#print(citations.head())

# Number of papers that cite or are cited
referenced_papers = list(set(list(citations[':START_ID']) + list(citations[':END_ID'])))
print(f'The total referenced papers are: {len((referenced_papers))}')

The total referenced papers are: 28590


In [4]:
# Merge cite and cite_has_citation

cite = pd.read_csv('DXML\\output_cite.csv',sep=';')
#print(cite.head())

# Merging
cited = pd.merge(citations, cite, left_on=':END_ID', right_on=':ID', how="left")
cited.drop(columns=[':ID'], inplace=True)

cited = cited.rename(columns={
    ':START_ID': ':ARTICLE_ID',
    ':END_ID': ':CITATION_ID'
})


cited.head()

#productID:ID(productID)
#supplierID:ID(supplierID)
#supplierID:ID(supplierID) --> :START_ID(supplierID) --> :END_ID(productID) --> productID:ID(productID)
#article:ID --> [:START_ID(:ARTICLE_ID) --> :END_ID(:CITATION_ID)] --> :ID[cite]

,:ARTICLE_ID,:CITATION_ID,cite:string
0,105324,13068502,reference/db/Boncz09c
1,105689,13068503,reference/db/LevineM09
2,106160,13068504,reference/db/Han09
3,106256,13068505,reference/db/Amati09c
4,6218246,13068506,books/bc/BatiniCN92


### ARTICLE NODE

In [5]:
# View total number of articles 

## Article headers
with open('DXML\\output_article_header.csv') as f:
    article_header = f.readline().split(';');
    #print(article_header[1:5])

## Number of articles in dataset
articles = pd.read_csv('DXML\\output_article.csv', header=0, names=article_header, sep=';');

## Display articles
#articles.head(2)

C:\Users\chibo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,14,18,19,20,21,22,23,24,25,26,28,30,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Pre-processing Total number of articles

## Drop articles columns with more than half null values 
articles.drop(columns = [i for i in articles.columns if (articles[i].isna().sum() > articles[i].count()/2) == True], inplace = True)

## Drop articles rows without author, number or articleID
articles.dropna(subset = [i for i in articles.columns], inplace = True)

In [7]:
# Select both cited and non-cited Articles randomly from data

## Selecting articles not in reference papers
articles_in_ref = articles[articles['article:ID'].isin(referenced_papers)]
print(f'The articles in list of reference papers are: {len((articles_in_ref))}')

## Selecting 200000 articles in reference papers
articles_not_in_ref = articles[~articles['article:ID'].isin(referenced_papers)].sample(n=200000, random_state=1)
print(f'The articles not in list of reference papers are: {len((articles_not_in_ref))}')

## Merging all selected articles together
articles_selected = pd.concat([articles_in_ref, articles_not_in_ref])
print(f'The total articles to be used are: {len((articles_selected))}')

## Display selected articles
display(articles_selected.head(3))

The articles in list of reference papers are: 1656
The articles not in list of reference papers are: 200000
The total articles to be used are: 201656


,article:ID,author:string[],ee:string[],journal:string,key:string,mdate:date,number:string,pages:string,title:string,url:string[],volume:string,year:int
673173,7285251,Dennis E. Shasha,http://sites.computer.org/debull/99june/dennisa.ps,IEEE Data Eng. Bull.,journals/debu/Shasha99,2020-03-10,2,40-46,Tuning Time Series Queries in Finance: Case Studies and Recommendations.,db/journals/debu/debu22.html#Shasha99,22.0,1999.0
673202,7285280,Alex Delis|Chung-Min Chen|Nick Roussopoulos|Stephen Kelley|Yannis Papakonstantinou,http://sites.computer.org/debull/95JUN-CD.pdf,IEEE Data Eng. Bull.,journals/debu/RoussopoulosCKDP95,2020-03-10,2,19-28,The ADMS Project: View R Us.,db/journals/debu/debu18.html#RoussopoulosCKDP95,18.0,1995.0
673216,7285294,Giansalvatore Mecca|Paolo Atzeni|Paolo Merialdo,http://sites.computer.org/debull/99sept/araneus.ps,IEEE Data Eng. Bull.,journals/debu/MeccaMA99,2020-03-10,3,19-26,Araneus in the Era of XML.,db/journals/debu/debu22.html#MeccaMA99,22.0,1999.0


In [8]:
#articles_selected.to_csv(path_or_buf='Mains/articles.csv',index=False)

In [9]:
# free up Memory Allocation

del articles
gc.collect()

18

### AUTHORS NODE

In [10]:
# View Authors

authors = pd.read_csv('DXML\\output_author.csv',sep=';')
authors_articles = pd.read_csv('DXML\\output_author_authored_by.csv', sep=';')

In [11]:
# Authors that have written articles
#authors_in_articles_sel = authors_articles[authors_articles[':START_ID'].isin(list(articles_selected['article:ID']))]

authors_in_articles_sel = pd.merge(authors_articles, articles_selected, left_on=':START_ID', 
                                   right_on='article:ID').drop(columns = [i for i in articles_selected.columns])

authors_in_articles_sel.shape

(614017, 2)

In [12]:
authors_in_articles_sel.head()

,:START_ID,:END_ID
0,6667623,9734488
1,8552975,9734490
2,8552975,9734852
3,8552975,10708262
4,8552975,11122363


In [13]:
authors.head()

,:ID,author:string
0,9734485,Oliver Hoffmann 0002
1,9734486,Florian Reitz
2,9734487,Russell Turpin
3,9734488,Frank Olken
4,9734489,Guido Frisch


In [14]:
# Authors that have written articles
authored_with_articles = pd.merge(authors, authors_in_articles_sel, left_on=':ID',
                                  right_on=':END_ID').drop(columns = [i for i in authors_in_articles_sel.columns])

authored_with_articles = authored_with_articles.drop_duplicates()

# Rename
#authored_with_articles = authored_with_articles.rename(columns={
#    ':START_ID': ':START_ID(:ARTICLE_ID)',
#    ':END_ID': ':END_ID(:AUTHOR_ID)'
#})

authored_with_articles.head()

#article:ID --> [:START_ID(:ARTICLE_ID) --> :END_ID(:CITATION_ID)] --> :ID[cite]

,:ID,author:string
0,9734488,Frank Olken
1,9734490,Stefano Cresci
5,9734498,Davide Tamborini
6,9734500,Filippo Bonchi
9,9734501,Alessia Amelio


In [15]:
authors_in_articles_sel.to_csv(path_or_buf='Mains/authors_publishes_articles.csv',index=False)
authored_with_articles.to_csv(path_or_buf='Mains/authors.csv',index=False)


In [16]:
print(authored_with_articles.shape)

del authors, authors_articles
gc.collect()

(378808, 2)


0

### JOURNALS NODE

In [17]:

#Similarly we want to keep only the journals for which we have articles published in
journals = pd.read_csv('DXML\\output_journal.csv',sep=';')
articles_in_journals = pd.read_csv('DXML\\output_journal_published_in.csv',sep=';')

In [18]:
journals_in_articles_sel = pd.merge(articles_in_journals, articles_selected, left_on=':START_ID', 
                                    right_on='article:ID').drop(columns = [i for i in articles_selected.columns])

journals_in_articles_sel.shape

(201656, 2)

In [19]:
journals_in_articles_sel.head()

,:START_ID,:END_ID
0,7270424,13088639
1,7270434,13088639
2,7270458,13088639
3,7270484,13088639
4,7270486,13088639


In [20]:
journals.head()

,:ID,journal:string
0,13088639,World Wide Web
1,13088640,SIGMOD Rec.
2,13088641,SIGMOD Record
3,13088642,EAI Endorsed Trans. Ubiquitous Environ.
4,13088643,Int. J. Trust. Manag. Comput. Commun.


In [21]:
merged_journals =  pd.merge(journals, journals_in_articles_sel, left_on=':ID', right_on=':END_ID').drop(columns = [i for i in journals_in_articles_sel.columns])


In [22]:
merged_journals.head()

,:ID,journal:string
0,13088639,World Wide Web
1,13088639,World Wide Web
2,13088639,World Wide Web
3,13088639,World Wide Web
4,13088639,World Wide Web


In [23]:
merged_journals.shape
#print(articles_sel_in_journals.nunique())
#print(articles_sel_in_journals.isna().sum())

del journals, articles_in_journals
gc.collect()

0

In [24]:
journals_in_articles_sel.to_csv(path_or_buf='Mains/articles_in_journal.csv',index=False)
merged_journals.to_csv(path_or_buf='Mains/journals.csv',index=False)


### INPROCEEDINGS - Conferences

In [25]:
#INPROCEEDINGS - Main

In [26]:
# View total number of Conferences

## Article headers
with open('DXML\\output_inproceedings_header.csv') as f:
    proceedings_header = f.readline().split(';');
    #print(proceedings_header[1:5])

## Number of articles in dataset
inproceedings = pd.read_csv('DXML\\output_inproceedings.csv', header=0, names=proceedings_header, sep=';');

## Display articles
#inproceedings.head(2)

C:\Users\chibo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,5,6,7,9,10,16,17,18,19,21,22,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
# Pre-processing Total number of inproceedings

## Drop inproceedings columns with more than half null values 
inproceedings.drop(columns = [i for i in inproceedings.columns if (inproceedings[i].isna().sum() > inproceedings[i].count()/2) == True], inplace = True)

## Drop inproceedings rows without author, number or articleID
inproceedings.dropna(subset = [i for i in inproceedings.columns], inplace = True)

In [28]:
print(inproceedings.shape)
inproceedings.head(2)

(2907284, 11)


,inproceedings:ID,author:string[],booktitle:string,crossref:string[],ee:string[],key:string,mdate:date,pages:string,title:string,url:string,year:int
0,3326528,Qiming Chen|Umeshwar Dayal,CoopIS,conf/coopis/2000,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,2017-05-24,311-322,Multi-Agent Cooperative Transactions for E-Commerce.,db/conf/coopis/coopis2000.html#ChenD00,2000
1,3326529,Emmanuel Cecchet|Renaud Lachaize|Takoua Abdellatif,CoopIS/DOA/ODBASE (2),conf/coopis/2004-2,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,2017-05-25,1571-1589,Evaluation of a Group Communication Middleware for Clustered J2EE Application Servers.,db/conf/coopis/coopis2004-2.html#AbdellatifCL04,2004


In [29]:
#Merge Inpro with Articles
articles_sel_in_inproceedings = pd.merge(articles_selected, inproceedings, left_on=['title:string','author:string[]'], right_on=['title:string', 'author:string[]'])

In [30]:
articles_sel_in_inproceedings.columns

Index(['article:ID', 'author:string[]', 'ee:string[]_x', 'journal:string',
       'key:string_x', 'mdate:date_x', 'number:string', 'pages:string_x',
       'title:string', 'url:string[]', 'volume:string', 'year:int_x',
       'inproceedings:ID', 'booktitle:string', 'crossref:string[]',
       'ee:string[]_y', 'key:string_y', 'mdate:date_y', 'pages:string_y',
       'url:string', 'year:int_y'],
      dtype='object')

In [31]:
sel_inproceedings = articles_sel_in_inproceedings.copy()


In [32]:
drop_col = ['author:string[]', 'ee:string[]_x', 'journal:string',
       'key:string_x', 'mdate:date_x', 'number:string', 'pages:string_x',
       'title:string', 'url:string[]', 'volume:string', 'year:int_x','booktitle:string', 'crossref:string[]',
       'ee:string[]_y', 'key:string_y', 'mdate:date_y', 'pages:string_y', 'url:string', 'year:int_y']

sel_inproceedings.drop(columns=drop_col, inplace = True)

# Rename
sel_inproceedings = sel_inproceedings.rename(columns={'article:ID': ':START_ID(:ARTICLE_ID)',
                                                                'inproceedings:ID': ':END_ID(:Inproceedings_ID)'})



In [33]:
sel_inproceedings.head()

,:START_ID(:ARTICLE_ID),:END_ID(:Inproceedings_ID)
0,7286206,4871121
1,7286249,5282688
2,7286673,5360887
3,8455374,6304288
4,8480582,3944520


In [34]:
drop_col = ['ee:string[]_x', 'journal:string', 'key:string_x', 'mdate:date_x', 'number:string', 'pages:string_x',
       'title:string', 'url:string[]', 'volume:string', 'year:int_x']

articles_sel_in_inproceedings.drop(columns=drop_col, inplace = True)

In [35]:
articles_sel_in_inproceedings.shape

(1635, 11)

In [36]:
sel_inproceedings.to_csv(path_or_buf='Mains/articles_in_conferences.csv',index=False)
articles_sel_in_inproceedings.to_csv(path_or_buf='Mains/conferences.csv',index=False)



In [37]:
del inproceedings
gc.collect()


0

### REVIEWERS NODE

In [38]:
# first we have to find the papers of each author.
art_auth_dict = defaultdict(list)
for idx,row in authors_in_articles_sel.iterrows():
    art_auth_dict[row[':START_ID']].append(row[':END_ID'])

In [39]:
all_papers = list(articles_selected['article:ID'])
all_authors = list(authored_with_articles[':ID'])

f1 = open('Mains\\article_reviews.csv', 'w', newline='')
f2 = open('Mains\\reviews.csv', 'w', newline='')
with open('Mains\\article_reviews.csv', 'w', newline='') as writeFile:
    writer1 = csv.writer(f1)
    writer2 = csv.writer(f2)
    writer1.writerow([':END_ID',':START_ID(:ARTICLE_ID)'])
    writer2.writerow([':END_ID','review:string','accepted:boolean',':START_ID(:ARTICLE_ID)'])
    for (i,paper) in enumerate(all_papers):
        if i % 1000 == 0:
            reviewers = random.sample(all_authors,1000)
#             print(i)
        # exclude authors for being selected as reviewers of a paper
        revs = set(reviewers) - set(art_auth_dict[paper])
        # Assumption: 3 reviewers per paper - all accept it
        revs = random.sample(revs,3)
        for r in revs:
            writer1.writerow([r,paper])
            writer2.writerow([r,lorem.sentence(),True,paper])

C:\Users\chibo\AppData\Local\Temp/ipykernel_12000/440580588.py:18: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  revs = random.sample(revs,3)


In [40]:
gc.collect()


0

## ADDING ABSTRACTS TO ARTICLES & CORRESPONDING AUTHOR

In [41]:
abstracts = []

for i in range(articles_selected.shape[0]):
    abstracts.append(lorem.paragraph())
        
articles_selected['abstract:string'] = abstracts

In [42]:
articles_selected['abstract:string'] = articles_selected[['abstract:string', 'title:string']].agg(' '.join, axis=1)

In [43]:
articles_selected["corresponding_author:string[]"] = articles_selected["author:string[]"].str.split("|", expand = True)[0]

In [44]:
articles_selected.head(1)

,article:ID,author:string[],ee:string[],journal:string,key:string,mdate:date,number:string,pages:string,title:string,url:string[],volume:string,year:int,abstract:string,corresponding_author:string[]
673173,7285251,Dennis E. Shasha,http://sites.computer.org/debull/99june/dennisa.ps,IEEE Data Eng. Bull.,journals/debu/Shasha99,2020-03-10,2,40-46,Tuning Time Series Queries in Finance: Case Studies and Recommendations.,db/journals/debu/debu22.html#Shasha99,22.0,1999.0,Dolorem dolorem non etincidunt etincidunt velit. Porro numquam sit quisquam ipsum numquam. Dolore voluptatem ipsum quaerat dolor sit. Ut amet ut est numquam. Amet voluptatem numquam dolorem labore tempora est magnam. Tuning Time Series Queries in Finance: Case Studies and Recommendations.,Dennis E. Shasha


In [45]:
articles_selected.to_csv(path_or_buf='Mains/articles.csv',index=False)

## KEYWORDS NODE

In [46]:
# KEYWORDS

split_title = articles_selected["title:string"].str.split(" ", expand=True)

kwords = []

for i in split_title[split_title.columns[0]].values.tolist():
    if len(i) > 15 :
        kwords.append(i)
        
my_list = list(set(list(kwords)))
print(my_list[4:8])

['Self-Configuration', 'Eagle-Eye-Inspired', 'Environment-Aware', 'In-Memory-Datenmanagement']


In [47]:
# Filter out words containing non-ASCII characters
my_list = [word for word in my_list if all(ord(char) < 128 for char in word)]

print(my_list[4:8])
len(my_list)

['Self-Configuration', 'Eagle-Eye-Inspired', 'Environment-Aware', 'In-Memory-Datenmanagement']


3391

In [48]:
db_keywords = ['data management', 'indexing', 'data modeling', 'big data', 'data processing'
               , 'data storage', 'data querying', 'semantic', 'database']

In [49]:
my_list
kw_ids = {}
i = 99999999
with open('Mains\\keywords.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow([':ID','word:string'])
    for kw in db_keywords+my_list:
        kw_ids[kw] = i
        writer.writerow([i,kw])
        i+=1
with open('Mains\\article_keywords.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    # to make sure we can test lab questions, we will make the subset of the papers that cite/are cited to 
    # belong to the database community
    writer.writerow([':START_ID',':END_ID'])
    for paper in referenced_papers:
        db_kw_num = random.randint(1,4)
        more_kw_num = random.randint(1,4)
        kws = random.sample(db_keywords,db_kw_num) + random.sample(my_list, more_kw_num)
        for kw in kws:
            writer.writerow([paper,kw_ids[kw]])
    for paper in (set(all_papers) - set(referenced_papers)):
        db_kw_num = random.randint(0,1)
        more_kw_num = random.randint(0,3)
        kws = random.sample(db_keywords,db_kw_num) + random.sample(my_list, more_kw_num)
        for kw in kws:
            writer.writerow([paper,kw_ids[kw]])

## AFFILIATION NODE

In [50]:
#Similarly we want to keep only the journals for which we have articles published in
school = pd.read_csv('DXML\\output_school.csv',sep=';')
output_school_submitted_at = pd.read_csv('DXML\\output_school_submitted_at.csv',sep=';')

In [51]:
school.head(2)

,:ID,school:string
0,13013659,"University of Texas, Austin"
1,13013660,University of California at Berkeley


In [52]:
output_school_submitted_at.head(2)

,:START_ID,:END_ID
0,1006,13013659
1,7,13013659


In [53]:
authors = list(authored_with_articles[':ID'])
all_schools = list(school[':ID'])

f1 = open('Mains\\author_affiliated_school.csv', 'w', newline='')
#f2 = open('Mains\\reviews.csv', 'w', newline='')
with open('Mains\\author_affiliated_school.csv', 'w', newline='') as writeFile:
    writer1 = csv.writer(f1)
    #writer2 = csv.writer(f2)
    writer1.writerow([':END_ID',':START_ID(:AUTHOR_ID)'])
    #writer2.writerow([':END_ID','review:string','accepted:boolean',':START_ID(:ARTICLE_ID)'])
    for (i,paper) in enumerate(authors):
        if i % 1000 == 0:
            reviewers = random.sample(all_schools,1000)
#             print(i)
        # exclude authors for being selected as reviewers of a paper
        revs = set(reviewers) - set(art_auth_dict[paper])
        # Assumption: 3 reviewers per paper - all accept it
        revs = random.sample(revs,1)
        for r in revs:
            writer1.writerow([r,paper])
            #writer2.writerow([r,lorem.sentence(),True,paper])

C:\Users\chibo\AppData\Local\Temp/ipykernel_12000/3065565245.py:18: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  revs = random.sample(revs,1)
